In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, split, udf
from pyspark.sql.types import IntegerType

In [7]:
spark = SparkSession.builder \
    .appName("Data Cleaning") \
    .getOrCreate()

In [8]:
input_path = "job_descriptions_30000.csv"
df = spark.read.csv(input_path, header=True, inferSchema=True)

In [9]:
def preprocess_experience(experience):
    years = experience.split(' to ')
    min_years = int(years[0])
    max_years = int(years[1].split(' ')[0])
    return (min_years + max_years) / 2

preprocess_experience_udf = udf(preprocess_experience, IntegerType())
df = df.withColumn("Experience", preprocess_experience_udf(col("Experience")))

In [10]:
df = df.withColumn("Salary Range", regexp_replace(col("Salary Range"), r'[^\d-]', ''))
df = df.withColumn("Salary Range", split(col("Salary Range"), '-').getItem(0).cast(IntegerType()))

In [11]:
output_path = "processed/job_descriptions_30000.csv"
df.write.csv(output_path, header=True)